<img src="https://zone.msn.com/images/v9/en-us/game/bljk/640x345_bljk.png">

### 1. IMPORTS AND GLOBAL VARIABLES:

In [7]:
import random

#### 1.1 GLOBAL VARIABLES

In [8]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
          'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

#### 1.2 VARIABLE FOR CONTROL FLOW IN WHILE LOOPS:

In [9]:
playing = True

### 2. CLASSES:

#### 2.1 CARD CLASS:

In [28]:
class Card():
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
    
    def __str__(self):
        return self.rank + ' of ' + self.suit

#### 2.2 DECK CLASS:

In [11]:
class Deck():
    
    def __init__(self):
        
        self.deck = []
        
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
    
    def __str__(self):
        
        deck_card_list = ''
        
        for card in self.deck:
            deck_card_list += '\n' + card.__str__()
        return 'The deck has: ' + deck_card_list
    
    def shuffle(self):
        random.shuffle(self.deck) # No return, as this should happen in-place.
    
    def deal(self):
        card = self.deck.pop()
        return card

In [16]:
# Testing the Deck Class

test_deck = Deck()
#print(test_deck)

In [17]:
# Testing the Deck Shuffle

test_deck.shuffle()
#print(test_deck)

#### 2.3 HAND CLASS:

In [26]:
# Represents what cards are in someone's hand (Dealer or Player). 
# Also, it will adjust the value of Aces when appropriate.

class Hand():
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0 # attribute to keep track of aces
    
    def add_card(self,card):
        self.cards.append(card) # card passed in is from Deck.deal(). Deck.deal() is a single card object Card(suit,rank)
        self.value += values[card.rank]
    
        if card.rank == 'Ace': # tracking for aces
            self.aces += 1     
    
    def adjust_for_ace(self):
        while self.value > 21 and self.aces: # 0 is going to be treated as False and other numbers not 0 are True.
            self.value -= 10
            self.aces -= 1

#### 2.4 CHIPS CLASS:

In [22]:
class Chips():
    
    def __init__(self,total=100):
        self.total = total # This can be set to a default value or supplied by a user input
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

### 3. FUNCTIONS:

#### 3.1 FUNCTION FOR TAKING BETS

In [29]:
# Since we are asking the player for an integer value, this would be a good place to use try/except.

def take_bet(chips):
    
    while True:
        
        try:
            chips.bet = int(input("\n How many chips would you like to bet? "))
        
        except:
            print("Sorry, please provide an integer")
        
        else: 
            if chips.bet > chips.total:
                print("Sorry, you do not have enough chips to bet! You have: {}".format(chips.total))
            else:
                break

#### 3.2 FUNCTION FOR TAKING HITS:

In [30]:
# This function will be called during gameplay when the player requests a hit or the dealer's hand is less than 17.
# Take in the Deck and Hand objects and 1 card dealt in the deck class will get added to the Hand and check for Aces.

def hit(deck,hand):
    
    popped_card = deck.deal()
    hand.add_card(popped_card)
    hand.adjust_for_ace()

#### 3.3 FUNCTION TO PROMPT THE PLAYER TO HIT OR STAND:

In [28]:
def hit_or_stand(deck,hand):
    
    global playing # to control an upcoming while loop
    
    while True:
        x = input("Hit or Stand? Enter h or s ")
        
        if x[0].lower() == 'h':
            hit(deck,hand)
        
        elif x[0].lower() == 's':
            print("Player Stands. Dealer's Turn ")
            playing = False
        
        else:
            print("Sorry, I did not understand that, Please enter h or s only!")
            continue
        
        break

#### 3.4 FUNCTION TO DISPLAY CARDS:

In [31]:
def show_some(player,dealer):
    print("\nDealer's Hand:")
    print(" <card hidden>")
    print('',dealer.cards[1])  
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    
def show_all(player,dealer):
    print("\nDealer's Hand:", *dealer.cards, sep='\n ')
    print("Dealer's Hand =",dealer.value)
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    print("Player's Hand =",player.value)

#### 3.5 FUNCTIONS FOR END OF GAME SCENARIOS:

In [32]:
def player_busts(player,dealer,chips):
    print("Player Busted!")
    chips.lose_bet()

def player_wins(player,dealer,chips):
    print("Player Wins!")
    chips.win_bet()

def dealer_busts(player,dealer,chips):
    print("Player Wins! Dealer Busted!")
    chips.win_bet()

def dealer_wins(player,dealer,chips):
    chips.lose_bet()

def push(player,dealer):
    print('Dealer and Player tie! PUSH')

### 4. GAME LOGIC:

In [34]:
while True:
    
    print("WELCOME TO BLACKJACK!")
    
    # Create deck, shuffle it, deal two cards to each player
    
    deck = Deck()
    deck.shuffle()
    
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # Set up player's chips
    
    player_chips = Chips()
    
    # Prompt player for their bet
    
    take_bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    
    show_some(player_hand,dealer_hand)
    
    while playing:
        
        # Prompt player to hit or stand
        
        hit_or_stand(deck,player_hand)
        
        # Show cards but keep one dealer card hidden
        
        show_some(player_hand,dealer_hand)
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,player_chips)
            
            break
    
    # If player hasn't busted, play Dealer's hand until Dealer reaches 17 (soft 17 strategy)
    
    if player_hand.value <= 21:
        
        while dealer_hand.value < 17:
            hit(deck,dealer_hand)
        
        # Show all cards
        show_all(player_hand,dealer_hand) 
        
        # Run different winning scenarios:
        
        if dealer_hand.value > 21:
            dealer_busts(player_hand, dealer_hand, player_chips)
            
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand, dealer_hand, player_chips)
            
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand,dealer_hand,player_chips)
            
        else:
            push(player_hand,dealer_hand)
            
    # Inform players of their remaining chips total:
    
    print("\n Player's total chips are at: {}".format(player_chips.total))
    
    # Ask if they want to play again
    
    new_game = input("Would you like to play another hand? y/n: ")
    
    if new_game[0].lower() == 'y':
        playing = True
        continue
        
    else:
        print("Thank you for playing!")
        break

WELCOME TO BLACKJACK!

 How many chips would you like to bet? 10

Dealer's Hand:
 <card hidden>
 Nine of Hearts

Player's Hand:
 King of Clubs
 Ten of Spades
Hit or Stand? Enter h or s s
Player Stands. Dealer's Turn 

Dealer's Hand:
 <card hidden>
 Nine of Hearts

Player's Hand:
 King of Clubs
 Ten of Spades

Dealer's Hand:
 Seven of Clubs
 Nine of Hearts
 King of Hearts
Dealer's Hand = 26

Player's Hand:
 King of Clubs
 Ten of Spades
Player's Hand = 20
Player Wins! Dealer Busted!

 Player's total chips are at: 110
Would you like to play another hand? y/n: y
WELCOME TO BLACKJACK!

 How many chips would you like to bet? 100

Dealer's Hand:
 <card hidden>
 Three of Hearts

Player's Hand:
 Three of Clubs
 Ace of Spades
Hit or Stand? Enter h or s h

Dealer's Hand:
 <card hidden>
 Three of Hearts

Player's Hand:
 Three of Clubs
 Ace of Spades
 Three of Diamonds
Hit or Stand? Enter h or s h

Dealer's Hand:
 <card hidden>
 Three of Hearts

Player's Hand:
 Three of Clubs
 Ace of Spades
 Three 